# Analysing and Quantifying the Intensity of ADF STEM Images

- 2018/3: First written by Katherine E. MacArthur 
- 2018/8: Minor improvements by Magnus Nord

## Step 1 : Import hyperspy and atomap.

In [ ]:
%matplotlib nbagg

In [ ]:
import numpy as np
import hyperspy.api as hs
import atomap.api as am

## Step 2: Import and analyse electron flux.

In [ ]:
s = hs.load('PACBED.tif')
s.plot()
flux_pattern = s.data

The following step creates an interactive plot to select the region which to fit a power law to the flux patter. Until anything is selected 'profiler' will be empty intially.

In [ ]:
profiler = am.quant.find_flux_limits(flux_pattern, 25)

Select the left limit with the left mouse button, the right limit with the right mouse button and then confirm with the Enter key. Good values here could be 103 for the left limit, and 406 for the right limit.

Once coordinates have been selected a message will be printed above confirming.

The limits can also be given directly to the function:

In [ ]:
profiler = am.quant.find_flux_limits(flux_pattern, 25, limits=(103, 406))

Running `analyse_flux` fits a power law to the flux profile in the selected region.

In [ ]:
expo = am.quant.analyse_flux(profiler[0].coords, profiler[1], 25)
expo

## Step 3: Import the an experimental image and a detector map.

In [ ]:
s = hs.load('Pt0.7Ni0.3_ADF.tif')
s.plot()
det_image = am.example_data.get_detector_image_signal()

## Step 4: Normalised images to an absolute scale.

For detector quantification with flux weighting use:

In [ ]:
s_normalised = am.quant.detector_normalisation(s, det_image, inner_angle=60, outer_angle=None, flux_expo=expo[0])
s_normalised.plot()

For detector quantification without flux weighting simply remove the flux exponent.

In [ ]:
s_normalised2 = am.quant.detector_normalisation(s, det_image, inner_angle=60, outer_angle=None)

## Step 5: Carry out peak finding on normalised image.

In [ ]:
s_peaks = am.get_feature_separation(s, separation_range=(5,30), threshold_rel=0.1)
s_peaks.plot()

In [ ]:
atom_positions = am.get_atom_positions(s, 8, threshold_rel=0.1)

In [ ]:
sublattice = am.Sublattice(atom_positions, s.data)
sublattice.plot()

In [ ]:
sublattice.construct_zone_axes()

In [ ]:
sublattice.refine_atom_positions_using_2d_gaussian(sublattice.image)

## Step 6: Carry out integration of intensity over each atomic column.

In [ ]:
(integrated_s, intensityRecord, pointRecord) = am.integrate(
    s, sublattice.x_position, sublattice.y_position, method='Voronoi', max_radius=13)

In [ ]:
intensityRecord.plot(cmap='viridis')

Please note no multiplication is added au